In [5]:
# !pip install scikit-optimize

# Mount Google Drive (for Google Colab)
#from google.colab import drive
#drive.mount('/content/drive')


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.8/107.8 kB 2.7 MB/s eta 0:00:00


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# imports for model selection and grid search
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from skopt import BayesSearchCV
from skopt.space import Real, Integer

from sklearn.multioutput import MultiOutputClassifier
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import BaggingClassifier

from sklearn.metrics import hamming_loss, accuracy_score, f1_score, classification_report

In [2]:
# Update file path to match Google Drive directory structure
#file_path = '/content/drive/MyDrive/learning and projects/DS/project/data_to_ML_model.csv'
#df = pd.read_csv(file_path, delimiter=",", low_memory=False)

Mounted at /content/drive


In [2]:
# Read the CSV file

file_path = r'C:\Users\ghiat\My Drive\learning and projects\DS\project\data_to_ML_model.csv'

df = pd.read_csv(file_path, delimiter=",", low_memory=False)

In [3]:
df.head()

,Returnees reported in the community,Three most common IDP shelter types - Solid finished apartment,Three most common IDP shelter types - Unfinished or abandoned residential building,Three most common IDP shelter types - Damaged residential building,Three most common IDP shelter types - Non residential structure with one family,Three most common IDP shelter types - Collective center with more than one family,Three most common IDP shelter types - Tent,Proportion of IDPs living in overcrowded shelters,IDP shelter inadequacies - Lack of lighting inside shelter,IDP shelter inadequacies - Lack of lighting around shelter,...,Livelihood support,Education,WASH,Winterisation,Legal services,GBV services,CP services,Explosive hazard risk awareness or removal of explosive contamination,Mental health psychological support,Cash assistance vouchers or cash in hand
0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.4,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [4]:
target_columns = [
    "Shelter",
    "Health",
    "NFIs",
    "Electricity assistance",
    "Food, nutrition",
    "Agricultural supplies",
    "Livelihood support",
    "Education",
    "WASH",
    "Winterisation",
    "Legal services",
    "GBV services",
    "CP services",
    "Explosive hazard risk awareness or removal of explosive contamination",
    "Mental health psychological support",
    "Cash assistance vouchers or cash in hand"
]

In [5]:
# Extract the target DataFrame
target = df[target_columns]

# df_feature = df.drop(columns=target.columns)
df_feature = df.drop(target_columns, axis=1)

In [6]:
for i in target.columns:
    print (i,target[i].value_counts(normalize=True).round(2))

Shelter Shelter
0.0    0.99
1.0    0.01
Name: proportion, dtype: float64
Health Health
0.0    0.76
1.0    0.24
Name: proportion, dtype: float64
NFIs NFIs
0.0    0.99
1.0    0.01
Name: proportion, dtype: float64
Electricity assistance Electricity assistance
0.0    0.99
1.0    0.01
Name: proportion, dtype: float64
Food, nutrition Food, nutrition
0.0    0.57
1.0    0.43
Name: proportion, dtype: float64
Agricultural supplies Agricultural supplies
0.0    0.99
1.0    0.01
Name: proportion, dtype: float64
Livelihood support Livelihood support
0.0    0.98
1.0    0.02
Name: proportion, dtype: float64
Education Education
0.0    0.97
1.0    0.03
Name: proportion, dtype: float64
WASH WASH
0.0    0.85
1.0    0.15
Name: proportion, dtype: float64
Winterisation Winterisation
0.0    0.98
1.0    0.02
Name: proportion, dtype: float64
Legal services Legal services
0.0    0.99
1.0    0.01
Name: proportion, dtype: float64
GBV services GBV services
0.0    1.0
1.0    0.0
Name: proportion, dtype: float64
CP s

In [7]:
X_train, X_test, Y_train, Y_test = train_test_split(df_feature, target, test_size=0.2, random_state=1234)

# Random Forest

In [8]:
# Create a nested ensemble: AdaBoost (boosting) of Bagging (bagging) of RandomForest
nested_rf = MultiOutputClassifier(
    AdaBoostClassifier(
        estimator=BaggingClassifier(
            estimator=RandomForestClassifier(random_state=1234, n_estimators=100, n_jobs=-1, class_weight='balanced'),
            n_estimators=10,
            random_state=1234
        ),
        n_estimators=50,
        random_state=1234,
        algorithm='SAMME'
    )
)

# Fit the model with training data
nested_rf.fit(X_train, Y_train)

# Predict and evaluate on the test set
Y_pred_rf = nested_rf.predict(X_test)
print("Boosting of Bagging RandomForest - Evaluation Metrics:")
print(f"Hamming Loss: {hamming_loss(Y_test, Y_pred_rf)}")
print(f"Exact Match Accuracy: {accuracy_score(Y_test, Y_pred_rf)}")
print("Classification Report:")
print(classification_report(Y_test, Y_pred_rf, target_names=target_columns, zero_division=0))

Boosting of Bagging RandomForest - Evaluation Metrics:
Hamming Loss: 0.022254291356524382
Exact Match Accuracy: 0.7514810876500075
Classification Report:
                                                                       precision    recall  f1-score   support

                                                              Shelter       1.00      0.06      0.12        31
                                                               Health       0.94      0.88      0.91      1638
                                                                 NFIs       1.00      0.13      0.24        67
                                               Electricity assistance       1.00      0.33      0.50        45
                                                      Food, nutrition       0.90      0.92      0.91      2862
                                                Agricultural supplies       0.00      0.00      0.00        47
                                                   Livelihood suppor

In [9]:
print("Samples with no true labels:", (Y_test.sum(axis=1) == 0).sum())
print("Samples with no predicted labels:", (Y_pred_rf.sum(axis=1) == 0).sum())

Samples with no true labels: 2739
Samples with no predicted labels: 3084


# Support Vector Machine

In [12]:
# Create a nested ensemble: AdaBoost (boosting) of Bagging (bagging) of LinearSVC (SVM)
nested_model = MultiOutputClassifier(
    AdaBoostClassifier(
        estimator=BaggingClassifier(
            estimator=LinearSVC(random_state=1234, C=1.0, max_iter=5000, class_weight='balanced'),
            n_estimators=10, random_state=1234),
        n_estimators=50, random_state=1234, algorithm='SAMME'
    )
)

# Fit the model with training data
nested_model.fit(X_train, Y_train)

# Predict and evaluate on the test set
Y_pred = nested_model.predict(X_test)
print("Boosting of Bagging SVM - Evaluation Metrics:")
print(f"Hamming Loss: {hamming_loss(Y_test, Y_pred)}")
print(f"Exact Match Accuracy: {accuracy_score(Y_test, Y_pred)}")
print("Classification Report:")
print(classification_report(Y_test, Y_pred, target_names=target_columns, zero_division=0))

/usr/local/lib/python3.11/dist-packages/sklearn/ensemble/_weight_boosting.py:519: FutureWarning: The parameter 'algorithm' is deprecated in 1.6 and has no effect. It will be removed in version 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/ensemble/_weight_boosting.py:519: FutureWarning: The parameter 'algorithm' is deprecated in 1.6 and has no effect. It will be removed in version 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/ensemble/_weight_boosting.py:519: FutureWarning: The parameter 'algorithm' is deprecated in 1.6 and has no effect. It will be removed in version 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/ensemble/_weight_boosting.py:519: FutureWarning: The parameter 'algorithm' is deprecated in 1.6 and has no effect. It will be removed in version 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/ensemble/_weight_boosting.py:519: FutureWarning: The parameter 'algorithm' is deprecated in 

Boosting of Bagging SVM - Evaluation Metrics:
Hamming Loss: 0.07052255810420781
Exact Match Accuracy: 0.5053926781102841
Classification Report:
                                                                       precision    recall  f1-score   support

                                                              Shelter       0.06      0.68      0.10        31
                                                               Health       0.80      0.76      0.78      1638
                                                                 NFIs       0.05      0.61      0.09        67
                                               Electricity assistance       0.19      0.73      0.30        45
                                                      Food, nutrition       0.82      0.90      0.86      2862
                                                Agricultural supplies       0.04      0.49      0.07        47
                                                   Livelihood support       0.

In [14]:
print("Samples with no true labels:", (Y_test.sum(axis=1) == 0).sum())
print("Samples with no predicted labels:", (Y_pred.sum(axis=1) == 0).sum())

Samples with no true labels: 2739
Samples with no predicted labels: 2643
